In [ ]:
#@title 📦 Download and install rclone binary automatically
import os
import zipfile
import shutil

# Download latest rclone release for Linux amd64
url = "https://downloads.rclone.org/rclone-current-linux-amd64.zip"
zip_path = "/content/rclone.zip"
extract_dir = "/content/rclone_extract"

# Download and unzip
print("⬇️ Downloading rclone...")
!curl -L -o "{zip_path}" "{url}"

print("📦 Extracting...")
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Move the rclone binary to ~/.local/bin
bin_path = os.path.expanduser("~/.local/bin")
os.makedirs(bin_path, exist_ok=True)

# Find the extracted rclone binary path
extracted_dir = next((d for d in os.listdir(extract_dir) if d.startswith("rclone-")), None)
rclone_binary_src = os.path.join(extract_dir, extracted_dir, "rclone")
rclone_binary_dest = os.path.join(bin_path, "rclone")

# Copy and set executable
shutil.copy2(rclone_binary_src, rclone_binary_dest)
os.chmod(rclone_binary_dest, 0o755)

# Update PATH for current session
%env PATH=/root/.local/bin:$PATH

# Confirm installation
print("✅ Rclone installed successfully at ~/.local/bin/rclone\n")
!rclone version


In [ ]:
#@title Upload your rclone.conf file
from google.colab import files
import os

print("📁 Upload your rclone.conf (found locally at ~/.config/rclone/rclone.conf)")
uploaded = files.upload()

config_dir = "/root/.config/rclone"
os.makedirs(config_dir, exist_ok=True)

for name in uploaded:
    if name.endswith(".conf"):
        os.rename(name, os.path.join(config_dir, "rclone.conf"))
        print(f"✔️ Uploaded and moved {name} to {config_dir}/rclone.conf")


In [ ]:
#@title Install dependencies
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent && apt install python3-libtorrent > /dev/null 2>&1



In [ ]:
#@title Enter the magnet link
import libtorrent as lt
import time
import datetime
import os

def choose_files(files):
    print("\n🔍 Torrent contains the following files:\n")
    for i, f in enumerate(files):
        print(f"[{i}] {f.path} ({f.size // (1024 * 1024)} MB)")

    selected = input("\nEnter indices of files to download (comma-separated), or 'all' to select all: ")

    if selected.strip().lower() == 'all':
        return list(range(len(files)))
    else:
        try:
            indices = [int(i.strip()) for i in selected.split(',')]
            return indices
        except:
            print("❌ Invalid input. Downloading all files.")
            return list(range(len(files)))

def download_and_upload(remote_name, remote_path):
    ses = lt.session()
    ses.listen_on(6881, 6891)

    # Set no download/upload rate limits for unrestricted speeds
    ses.set_download_rate_limit(0)  # Unlimited download rate
    ses.set_upload_rate_limit(0)  # Unlimited upload rate (this can be kept or disabled)

    # Optional: Limit the number of connections and uploads to optimize performance
    ses.set_max_connections(200)  # Increase max connections
    ses.set_max_uploads(10)  # Limit the number of upload connections (optional)

    params = {
        'save_path': '/content/Downloads/',
        'storage_mode': lt.storage_mode_t(2),
    }

    magnet = input("🔗 Enter magnet link: ").strip()
    handle = lt.add_magnet_uri(ses, magnet, params)
    handle.set_sequential_download(False)
    ses.start_dht()

    print("\n⏳ Fetching metadata...")
    while not handle.has_metadata():
        time.sleep(1)

    torrent_info = handle.get_torrent_info()
    files = torrent_info.files()

    selected_indices = choose_files(files)

    for idx in range(len(files)):
        if idx not in selected_indices:
            handle.file_priority(idx, 0)

    print(f"\n✅ Starting download: {torrent_info.name()}")
    print(datetime.datetime.now())

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        state_str = ['queued', 'checking', 'downloading metadata',
                     'downloading', 'finished', 'seeding', 'allocating']
        print(f"{s.progress * 100:.2f}% complete (↓ {s.download_rate/1000:.1f} kB/s | ↑ {s.upload_rate/1000:.1f} kB/s | peers: {s.num_peers}) [{state_str[s.state]}]")
        time.sleep(5)

    print(f"\n🎉 Download complete: {handle.name()}")
    print("📤 Uploading to rclone remote...")

    # Remote path like: myremote:path/in/remote
    full_remote_path = f"{remote_name}:{remote_path.rstrip('/')}/{handle.name()}"

    !rclone copy "/content/Downloads/{handle.name()}" "{full_remote_path}" --quiet

    print("✅ Upload finished!")

# Usage
remote = input("🌐 Enter your rclone remote name (e.g., gdrive): ").strip()
rpath = input("📁 Enter path in remote (e.g., Torrents/Anime): ").strip()
download_and_upload(remote, rpath)
